In [1]:
import os

import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb

import pandas as pd
import numpy as np
import multiprocessing
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc
from time import time
import datetime
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score


import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR

from sklearn.metrics import mean_absolute_error as sk_mean_absolute_error
from sklearn.metrics import roc_auc_score as sk_roc_auc_score
import numpy as np
import pandas as pd

import eli5
from eli5.sklearn import PermutationImportance

warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

In [2]:
folder_path = '../data/IEEE-CIS-Fraud-Detection/'

In [4]:
df_train = pd.read_pickle(f'{folder_path}/df_train3.gzde', compression='gzip')#.iloc[:10000,:]
df_test = pd.read_pickle(f'{folder_path}/df_test3.gzde', compression='gzip')#.iloc[:10000,:]

In [12]:
df_train = df_train.replace([np.inf, -np.inf], np.nan).fillna(-999)

df_test = df_test.replace([np.inf, -np.inf], np.nan).fillna(-999)

In [13]:
columns = df_train.columns.tolist()
# columns.remove('TransactionID')
# columns.remove('TransactionDT')
columns.remove('isFraud')

In [14]:
params = {
    'n_estimators':500,
    'max_depth':9,
    'learning_rate':0.05,
    'subsample':0.9,
    'colsample_bytree':0.9,
    'missing':-999,
    'random_state':2019,
    'tree_method':'gpu_hist'  # THE MAGICAL PARAMETER
}

In [15]:
def process(params, folds, df_train,df_test, columns, is_output_feature_importance=1, verbose=0):

#     aucs = list()
    his = []
    training_start_time = time()
    df_feature_importances_i_list = []
    df_valid_pred = pd.DataFrame()
    df_test_pred = pd.DataFrame()
    if type(df_test) != type(None):
        df_test_pred['TransactionID'] = df_test['TransactionID']
    
    X,y = df_train.sort_values('TransactionDT')[columns], df_train.sort_values('TransactionDT')['isFraud']
    if type(df_test) != type(None):
        X_test = df_test.sort_values('TransactionDT')[columns]
        
    for fold, (trn_idx, test_idx) in enumerate(folds.split(X, y)):
        start_time = time()
        if verbose > 1:
            print('Training on fold {}'.format(fold + 1))
        
        clf = xgb.XGBClassifier(**params)
        clf.fit(X.iloc[trn_idx].values, y.iloc[trn_idx].values)
        
        y_trn_pred = clf.predict_proba(X.iloc[trn_idx].values)[:,1]        
        y_val_pred = clf.predict_proba(X.iloc[test_idx].values)[:,1]
        
        original_index = df_train['TransactionID'].values[test_idx]
        df_valid_pred_i = pd.DataFrame({'TransactionID': original_index, 'predict': y_val_pred, 'fold': np.zeros(y_val_pred.shape[0]) + fold})
        df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
        
        y_test_pred = None
        if type(df_test)!=type(None):
            y_test_pred = clf.predict_proba(X_test.values)[:,1]
            df_test_pred_i = pd.DataFrame({fold: y_test_pred})
            df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=1)
        
        trn_auc = roc_auc_score(y.iloc[trn_idx].values, y_trn_pred)
        val_auc = roc_auc_score(y.iloc[test_idx].values, y_val_pred)
        
        if is_output_feature_importance:
            perm = PermutationImportance(clf, random_state=42).fit(X.iloc[test_idx].values, y.iloc[test_idx].values)
            df_feature_importances_i2 = eli5.explain_weights_dfs(perm, feature_names=columns, top=len(columns))['feature_importances']
            df_feature_importances_i2 = df_feature_importances_i2.sort_values(by=['feature'])
            df_feature_importances_i2 = df_feature_importances_i2.reset_index(drop=True)
            df_feature_importances_i_list.append(df_feature_importances_i2)
        
#         aucs.append(clf.best_score['valid_1']['auc'])
        his.append({'val_auc':val_auc, 'trn_auc':trn_auc, 'y_val_pred':y_val_pred, 'y_test_pred':y_test_pred, 'test_idx':test_idx})
        if verbose > 0:
            print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))
    
    his = pd.DataFrame(his)
    
    df_feature_importances = None
    if is_output_feature_importance:
        df_feature_importances = df_feature_importances_i_list[0]
        for idx, df_feature_importances_i in enumerate(df_feature_importances_i_list[1:]):
            df_feature_importances = pd.merge(df_feature_importances, df_feature_importances_i, on='feature', suffixes=('', idx + 1))
            
    df_valid_pred = df_valid_pred.sort_values(by=['TransactionID'])
    df_valid_pred = df_valid_pred.reset_index(drop=True)

    if type(df_test) != type(None):
        df_test_pred = df_test_pred.sort_values(by=['TransactionID'])
        df_test_pred = df_test_pred.reset_index(drop=True)
    
    if verbose > 0:
        print('-' * 30)
        print('Training has finished.')
        print('Total training time is {}'.format(str(datetime.timedelta(seconds=time() - training_start_time))))
        print('Mean AUC:', his.val_auc.mean(), his.trn_auc.mean())
        print('-' * 30)
    return his, df_feature_importances, df_valid_pred, df_test_pred, his.val_auc.mean()

In [19]:
columns_emd_over_o4 = ['C7_fq_enc',
 'ProductCD_target_mean',
 'C4_fq_enc',
 'id_35',
 'addr2_fq_enc',
 'C12_fq_enc',
 'D7_fq_enc',
 'V94',
 'almost-com_addr2',
 'addr2',
 'V200',
 'V201',
 'TransactionAmt_to_mean_addr1',
 'id_17',
 'C8_fq_enc',
 'ProductCD',
 'V171',
 'card3_TransactionAmt_std',
 'C10_fq_enc',
 'V170',
 'TransactionAmt_to_std_addr1',
 'ieee-gb-2-_id_33',
 'email_check',
 'card3_fq_enc',
 'card3_count_full',
 'id_29',
 'id_28',
 'V194',
 'V242',
 'V189',
 'R_emaildomain_suffix',
 'V188',
 'V186',
 'feature-en_R_emaildomain_suffix',
 'extensive-_R_emaildomain_suffix',
 'M4_target_mean',
 'eda-and-mo_addr2',
 'D6_fq_enc',
 'V244',
 'extensive-_PCA_V_17',
 'PCA_V_17',
 'eda-and-mo_card3',
 'D14',
 'V184',
 'R_emaildomain_2',
 'id_38',
 'card3',
 'id_09',
 'id_10',
 'ieee-gb-2-_D9',
 'ieee-gb-2-_id_29',
 'D8_fq_enc',
 'V197',
 'id_16',
 'id_15',
 'D12',
 'V239',
 'V195',
 'V185',
 'id_30_version',
 'id_01_count_dist',
 'V198',
 'id_03',
 'id_20',
 'V238',
 'ieee-gb-2-_id_16',
 'almost-com_R_emaildomain',
 'DeviceInfo_version_fq_enc',
 'V243',
 'id_31_device',
 'eda-and-mo_id_02_to_mean_card4',
 'id_02_to_mean_card4',
 'eda-and-mo_id_02_to_std_card4',
 'id_02_to_std_card4',
 'id_04',
 'id_02',
 'V199',
 'V169',
 'id_36',
 'ieee-gb-2-_id_31',
 'D7',
 'V190',
 'V176',
 'V191',
 'D15_to_mean_addr2',
 'id_02_to_mean_card1',
 'eda-and-mo_id_02_to_mean_card1',
 'id_02_to_std_card1',
 'eda-and-mo_id_02_to_std_card1',
 'id_31_count_dist',
 'DeviceInfo_fq_enc',
 'DeviceInfo_device_fq_enc',
 'V196',
 'id_11',
 'eda-and-mo_id_11',
 'lgb-single_id_11',
 'D15_to_mean_addr1',
 'eda-and-mo_D15_to_mean_addr1',
 'V220',
 'id_05',
 'V251',
 'V181',
 'V222',
 'V193',
 'V187',
 'lgb-single_V208',
 'eda-and-mo_V208',
 'V208',
 'R_emaildomain_fq_enc',
 'D15_to_std_addr1',
 'id_31_device_fq_enc',
 'V259',
 'V260',
 'V174',
 'eda-and-mo_V214',
 'lgb-single_V214',
 'V214',
 'V216',
 'eda-and-mo_V216',
 'lgb-single_V216',
 'eda-and-mo_V202',
 'lgb-single_V202',
 'V202',
 'eda-and-mo_V215',
 'V215',
 'lgb-single_V215',
 'eda-and-mo_V211',
 'V211',
 'lgb-single_V206',
 'eda-and-mo_V206',
 'V206',
 'eda-and-mo_V213',
 'lgb-single_V213',
 'V213',
 'lgb-single_V204',
 'V204',
 'eda-and-mo_V204',
 'eda-and-mo_V205',
 'lgb-single_V205',
 'V205',
 'eda-and-mo_V210',
 'lgb-single_V210',
 'V210',
 'eda-and-mo_V212',
 'lgb-single_V212',
 'V212',
 'V221',
 'eda-and-mo_V207',
 'lgb-single_V207',
 'V207',
 'V183',
 'V302',
 'V250',
 'lgb-single_V203',
 'V203',
 'eda-and-mo_V203',
 'eda-and-mo_id_19',
 'V235',
 'V262',
 'V192',
 'id_36_count_dist',
 'V247',
 'V209',
 'lgb-single_V209',
 'eda-and-mo_V209',
 'V245',
 'V249',
 'id_36_count_full',
 'V256',
 'V258',
 'V180',
 'V257',
 'V172',
 'V229',
 'V304',
 'V175',
 'V255',
 'V252',
 'V167',
 'V177',
 'eda-and-mo_V271',
 'lgb-single_V271',
 'V271',
 'lgb-single_V272',
 'eda-and-mo_V272',
 'V272',
 'browser_id_31',
 'V230',
 'lgb-single_V270',
 'eda-and-mo_V270',
 'V270',
 'V227',
 'id_06',
 'V182',
 'V179',
 'V168',
 'V246',
 'V79',
 'V173',
 'V248',
 'V178',
 'V228',
 'V232',
 'D13',
 'id_13',
 'V218',
 'V261',
 'ieee-gb-2-_ProductCD',
 'V219',
 'V237',
 'V217',
 'V233',
 'id_31',
 'V236',
 'V231',
 'V123',
 'V254',
 'id_30_device_fq_enc',
 'V234',
 'D15_to_std_addr2',
 'V57',
 'ieee-gb-2-_id_37',
 'V253',
 'D6',
 'V264',
 'lgb-single_V264',
 'eda-and-mo_V264',
 'eda-and-mo_V265',
 'lgb-single_V265',
 'V265',
 'lgb-single_V274',
 'eda-and-mo_V274',
 'V274',
 'eda-and-mo_V275',
 'lgb-single_V275',
 'lgb-single_V263',
 'V263',
 'eda-and-mo_V263',
 'lgb-single_V273',
 'eda-and-mo_V273',
 'V226',
 'lgb-single_V276',
 'eda-and-mo_V276',
 'V276',
 'V225',
 'lgb-single_V277',
 'eda-and-mo_V277',
 'V277',
 'id_30_fq_enc',
 'V278',
 'eda-and-mo_V278',
 'lgb-single_V278',
 'V240',
 'id_33_fq_enc',
 'V241',
 'eda-and-mo_V268',
 'lgb-single_V268',
 'V268',
 'eda-and-mo_V266',
 'lgb-single_V266',
 'V266',
 'eda-and-mo_V269',
 'V269',
 'V267',
 'eda-and-mo_V267',
 'lgb-single_V267',
 'id_30_version_fq_enc',
 'V224',
 'V223',
 'V303',
 'V93',
 'almost-com_M6',
 'id_01',
 'card3_TransactionAmt_mean',
 'V92',
 'V71',
 'id_37',
 'id_12',
 'DeviceInfo_version',
 'eda-and-mo_id_13',
 'V21',
 'DeviceType',
 'id_19',
 'V74',
 'id_02__id_20',
 'addr1_fq_enc',
 'id_02__D8',
 'PCA_V_28',
 'lgb-single_D9',
 'eda-and-mo_D9',
 'D9',
 'lgb-single_D8',
 'eda-and-mo_D8',
 'D8',
 'addr1',
 'extensive-_PCA_V_28',
 'feature-en_R_emaildomain_bin',
 'extensive-_R_emaildomain_bin',
 'almost-com_addr1',
 'V125',
 'C5_fq_enc',
 'V58',
 'V73',
 'eda-and-mo_addr1',
 'V72',
 'ieee-gb-2-_id_28',
 'almost-com_ProductCD',
 'V273',
 'V69',
 'R_emaildomain',
 'V64',
 'extensive-_PCA_V_19',
 'PCA_V_19',
 'V283',
 'card6_count_full',
 'ieee-gb-2-_card6',
 'card6',
 'M6',
 'V275',
 'V10',
 'id_33_0',
 'V11',
 'V63',
 'PCA_V_8',
 'extensive-_PCA_V_8',
 'PCA_V_23',
 'extensive-_PCA_V_23',
 'id_33_1',
 'addr1__card1',
 'device_version',
 'device_name',
 'PCA_V_34',
 'first_value_addr1',
 'V29',
 'PCA_V_1',
 'M_na',
 'PCA_V_31',
 'ieee-gb-2-_id_15',
 'dist2_fq_enc',
 'M_sum',
 'V85',
 'V70',
 'M1',
 'almost-com_M1',
 'ieee-gb-2-_M1',
 'V90',
 'ieee-gb-2-_M2',
 'dist2',
 'V1',
 'D11',
 'V22',
 'V6',
 'V9',
 'V111',
 'V8',
 'R_emaildomain_1',
 'V3',
 'V2',
 'V7',
 'eda-and-mo_id_20',
 'V5',
 'V4',
 'V113',
 'almost-com_M4',
 'PCA_V_30',
 'ieee-gb-2-_M3',
 'V84',
 'card2_TransactionAmt_mean',
 'PCA_V_29',
 'C2_fq_enc',
 'V112',
 'V114',
 'V282',
 'V18',
 'PCA_V_3',
 'V60',
 'V50',
 'V17',
 'extensive-_PCA_V_29',
 'V54',
 'C11_fq_enc',
 'V30',
 'ieee-gb-2-_DeviceInfo',
 'V59',
 'V33',
 'DeviceInfo_device',
 'id_18',
 'C1_fq_enc',
 'DeviceInfo',
 'V53',
 'DeviceInfo__P_emaildomain',
 'V15',
 'D2',
 'V124',
 'M2',
 'extensive-_PCA_V_25',
 'extensive-_PCA_V_26',
 'PCA_V_26',
 'PCA_V_25',
 'V108',
 'PCA_V_22',
 'D11__DeviceInfo',
 'extensive-_PCA_V_22',
 'V91',
 'V31',
 'D15_to_std_card4',
 'eda-and-mo_D15_to_std_card4',
 'PCA_V_4',
 'extensive-_PCA_V_4',
 'V116',
 'version_id_31',
 'D10',
 'PCA_V_11',
 'extensive-_PCA_V_11',
 'V81',
 'M3',
 'D15_to_mean_card4',
 'eda-and-mo_D15_to_mean_card4',
 'id_30_device',
 'almost-com_M7',
 'D15',
 'PCA_V_7',
 'extensive-_PCA_V_7',
 'M7',
 'ieee-gb-2-_M9',
 'ieee-gb-2-_M6',
 'V13',
 'dist1_fq_enc',
 'V110',
 'D1',
 'V80',
 'V12',
 'extensive-_PCA_V_12',
 'PCA_V_12',
 'V67',
 'V51',
 'dist1',
 'eda-and-mo_dist1',
 'lgb-single_dist1',
 'PCA_V_0',
 'V76',
 'PCA_V_33',
 'card1_TransactionAmt_std',
 'extensive-_PCA_V_10',
 'PCA_V_10',
 'M8',
 'V281',
 'V43',
 'PCA_V_27',
 'V75',
 'uid2_TransactionAmt_std',
 'ieee-gb-2-_R_emaildomain',
 'card2_TransactionAmt_std',
 'uid_TransactionAmt_std',
 'extensive-_PCA_V_27',
 'R_emaildomain_prefix',
 'M9',
 'V48',
 'V34',
 'V49',
 'V66',
 'V16',
 'D2_fq_enc',
 'V23',
 'PCA_V_15',
 'extensive-_PCA_V_15',
 'V56',
 'V32']

In [20]:
folds = TimeSeriesSplit(n_splits=5)

In [21]:
trial = []

In [ ]:
his, df_feature_importances, df_valid_pred, df_test_pred, val_metric = process(params, folds, df_train,None, columns_emd_over_o4, is_output_feature_importance=1, verbose=1)

In [ ]:
def sort_feature_importances(df_feature_importances, key='average_permutation_weight'):
    df_feature_importances['average_permutation_weight'] = df_feature_importances[
        [col for col in df_feature_importances.columns.tolist() if ('weight' in col) & ('model' not in col)]].mean(
        axis=1)
    df_feature_importances = df_feature_importances.sort_values(by=[key], ascending=False)
    sorted_columns = df_feature_importances.feature.tolist()
    return sorted_columns

In [ ]:
sorted_columns = sort_feature_importances(df_feature_importances)

In [ ]:
sorted_columns

In [15]:
def add_feature_obo(folds, df_train, columns, max_col = 50):
    selected_columns = [columns[0]]
    if type(max_col) == type(None):
        max_col = len(columns)
    best_val_metric = 0
    for col in columns[1:]:
        his, df_feature_importances, df_valid_pred, df_test_pred, val_metric = process(folds, df_train,None, selected_columns + [col], is_output_feature_importance=0, verbose=0)
        if val_metric > best_val_metric:
            selected_columns = selected_columns + [col]
            best_val_metric = val_metric
        if len(selected_columns) > max_col:
            break
    return selected_columns

In [16]:
select_columns = add_feature_obo(folds, df_train, sorted_columns, max_col = 50)

In [7]:
select_columns = ['PCA_C_1',
 'C13',
 'C14',
 'C1_fq_enc',
 'PCA_C_2',
 'V70',
 'D2',
 'M4_target_mean',
 'V317',
 'uid3_TransactionAmt_mean',
 'card3_TransactionAmt_mean',
 'R_emaildomain',
 'C11',
 'D15_to_mean_addr1',
 'V258',
 'uid3_TransactionAmt_std',
 'card2_fq_enc',
 'card2_TransactionAmt_mean',
 'addr1__card1',
 'TransactionAmt',
 'ProductCD_target_mean',
 'uid3_fq_enc',
 'M6',
 'card5__P_emaildomain',
 'C5',
 'C2',
 'D3',
 'P_emaildomain',
 'V283',
 'card2',
 'V45',
 'uid_TransactionAmt_std',
 'TransactionAmt_decimal',
 'V189',
 'card1_fq_enc',
 'card1_count_full',
 'card3',
 'card6_count_full',
 'V281',
 'device_version',
 'V49',
 'V219',
 'V160',
 'DeviceInfo_fq_enc',
 'V224',
 'V220',
 'R_emaildomain_fq_enc',
 'id_02_to_std_card4']

In [8]:
len(select_columns)

48

In [9]:
folds = KFold(n_splits=8, shuffle=False, random_state=42)

In [10]:
his, df_feature_importances, df_valid_pred, df_test_pred, val_metric = process(params, folds, df_train,df_test, select_columns, is_output_feature_importance=0, verbose=1)

Fold 1 finished in 0:00:12.187674
Fold 2 finished in 0:00:11.135797
Fold 3 finished in 0:00:10.692355
Fold 4 finished in 0:00:10.777341
Fold 5 finished in 0:00:10.839463
Fold 6 finished in 0:00:10.683155
Fold 7 finished in 0:00:11.233347
Fold 8 finished in 0:00:10.709949
------------------------------
Training has finished.
Total training time is 0:01:30.680419
Mean AUC: 0.9384486843316294 0.9939956094743174
------------------------------


In [11]:
his

,val_auc,trn_auc,y_val_pred,y_test_pred,test_idx
0,0.923146,0.993820,"[0.0077880863, 0.010555981, 0.012954445, 0.002...","[0.0003591986, 0.0025347471, 0.00040779446, 0....","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,0.934077,0.994133,"[0.0024849277, 0.000413746, 0.003861929, 0.022...","[0.00042604352, 0.0035482023, 0.00022841957, 0...","[73818, 73819, 73820, 73821, 73822, 73823, 738..."
2,0.940063,0.993973,"[0.0010109397, 0.00085254107, 0.17099088, 0.00...","[0.00031060696, 0.0015484462, 0.00031299517, 0...","[147636, 147637, 147638, 147639, 147640, 14764..."
3,0.948651,0.993887,"[0.0042435494, 0.0014896975, 0.011838425, 0.21...","[0.00023208506, 0.00253229, 0.00083499117, 0.0...","[221454, 221455, 221456, 221457, 221458, 22145..."
4,0.930333,0.994051,"[0.024144322, 0.010336549, 7.8937934e-05, 0.00...","[0.00043045884, 0.0033303432, 0.0007339707, 0....","[295272, 295273, 295274, 295275, 295276, 29527..."
5,0.952062,0.994030,"[0.056929596, 0.0054036444, 0.0012144378, 0.00...","[0.0003165709, 0.0026185215, 0.000590887, 0.00...","[369089, 369090, 369091, 369092, 369093, 36909..."
6,0.947755,0.993797,"[0.00024381983, 0.00057942915, 0.005027751, 0....","[0.00017454267, 0.0025454902, 0.0004474988, 0....","[442906, 442907, 442908, 442909, 442910, 44291..."
7,0.931501,0.994274,"[0.004841661, 0.00013822324, 0.022244854, 0.08...","[0.00042076828, 0.0027218536, 0.00029605618, 0...","[516723, 516724, 516725, 516726, 516727, 51672..."


In [15]:
df_valid_pred.to_pickle(f'{folder_path}/oof_xgbm_train', compression='gzip')

In [16]:
df_test_pred.to_pickle(f'{folder_path}/oof_xgbm_test', compression='gzip')

In [14]:
df_test_pred.head()

,TransactionID,0,1,2,3,4,5,6,7
0,3663549,0.000359,0.000426,0.000311,0.000232,0.000430,0.000317,0.000175,0.000421
1,3663550,0.002535,0.003548,0.001548,0.002532,0.003330,0.002619,0.002545,0.002722
2,3663551,0.000408,0.000228,0.000313,0.000835,0.000734,0.000591,0.000447,0.000296
3,3663552,0.001834,0.001365,0.001245,0.001317,0.002828,0.001607,0.001286,0.001984
4,3663553,0.000255,0.000267,0.000337,0.000523,0.000307,0.000524,0.000452,0.000647


In [8]:
his, df_feature_importances, df_valid_pred, df_test_pred, val_metric = process(folds, df_train,None, columns, is_output_feature_importance=0, verbose=1)

Fold 1 finished in 0:00:16.474434
Fold 2 finished in 0:00:24.685806
Fold 3 finished in 0:00:33.262098
Fold 4 finished in 0:00:41.616481
Fold 5 finished in 0:00:50.096428
------------------------------
Training has finished.
Total training time is 0:02:48.768516
Mean AUC: 0.9151669630621226 0.9992160669360868
------------------------------


In [22]:
df_test = pd.read_pickle(f'{folder_path}/df_test2.gzde', compression='gzip')

In [18]:
folds = KFold(n_splits=8, shuffle=False, random_state=42)

In [23]:
his, df_feature_importances, df_valid_pred, df_test_pred, val_metric = process(folds, df_train,df_test, select_columns, is_output_feature_importance=0, verbose=1)

Fold 1 finished in 0:00:11.631914
Fold 2 finished in 0:00:11.586418
Fold 3 finished in 0:00:11.236524
Fold 4 finished in 0:00:11.626258
Fold 5 finished in 0:00:11.453612
Fold 6 finished in 0:00:11.261663
Fold 7 finished in 0:00:11.360538
Fold 8 finished in 0:00:11.525885
------------------------------
Training has finished.
Total training time is 0:01:34.385489
Mean AUC: 0.9384486843316294 0.9939956094743174
------------------------------


In [24]:
df_valid_pred

,TransactionID,predict,fold
0,2987000,0.007788,0.0
1,2987001,0.010556,0.0
2,2987002,0.012954,0.0
3,2987003,0.002685,0.0
4,2987004,0.001727,0.0
...,...,...,...
590535,3577535,0.002428,7.0
590536,3577536,0.003550,7.0
590537,3577537,0.002517,7.0
590538,3577538,0.016379,7.0


In [27]:
df_test_pred['predict'] = df_test_pred[np.arange(8)].mean(axis=1)

In [28]:
df_test_pred

,TransactionID,0,1,2,3,4,5,6,7,predict
0,3663549,0.000359,0.000426,0.000311,0.000232,0.000430,0.000317,0.000175,0.000421,0.000334
1,3663550,0.002535,0.003548,0.001548,0.002532,0.003330,0.002619,0.002545,0.002722,0.002672
2,3663551,0.000408,0.000228,0.000313,0.000835,0.000734,0.000591,0.000447,0.000296,0.000482
3,3663552,0.001834,0.001365,0.001245,0.001317,0.002828,0.001607,0.001286,0.001984,0.001683
4,3663553,0.000255,0.000267,0.000337,0.000523,0.000307,0.000524,0.000452,0.000647,0.000414
...,...,...,...,...,...,...,...,...,...,...
506686,4170235,0.007447,0.010083,0.008104,0.008997,0.006112,0.012237,0.009779,0.014781,0.009693
506687,4170236,0.004837,0.007242,0.006549,0.005499,0.006661,0.007814,0.008053,0.004442,0.006387
506688,4170237,0.015303,0.012308,0.014801,0.016541,0.011299,0.012146,0.018490,0.015931,0.014602
506689,4170238,0.017460,0.020305,0.010282,0.024031,0.018472,0.020822,0.013550,0.019312,0.018029


In [ ]:
params = {
    'n_estimators':500,
    'max_depth':9,
    'learning_rate':0.05,
    'subsample':0.9,
    'colsample_bytree':0.9,
    'missing':-999,
    'random_state':2019,
    'tree_method':'gpu_hist'  # THE MAGICAL PARAMETER
}

In [34]:
import optuna
#  tune hypterparameters
def objective(trial):
        
    max_depth = trial.suggest_int('max_depth', 3, 9)
    max_bin = trial.suggest_int('max_bin', 10, 100)
    eta = trial.suggest_uniform('eta', 0.01, 0.4)
    colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.6, 1)
    min_child_weight = trial.suggest_int('min_child_weight', 200, 600)
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    subsample = trial.suggest_uniform('subsample', 0.6, 1.0)
    reg_lambda = trial.suggest_uniform('reg_lambda', 0.000001, 1.0)
    reg_alpha = trial.suggest_uniform('reg_alpha', 0.000001, 1.0)
#     random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        "max_depth":max_depth,
        "max_bin":max_bin,
        "eta":eta,
        "colsample_bytree":colsample_bytree,
        "min_child_weight":min_child_weight,
        "n_estimators":n_estimators,
        "subsample":subsample,
        "reg_lambda":reg_lambda,
        "reg_alpha":reg_alpha,
        'tree_method':'gpu_hist',
    }
    
    his, df_feature_importances, df_valid_pred, df_test_pred, val_metric = process(args, folds, df_train,None, select_columns, is_output_feature_importance=0, verbose=0)

    return 1-val_metric

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-09-28 05:23:56,324] Finished trial#0 resulted in value: 0.10693959156734256. Current best value is 0.10693959156734256 with parameters: {'max_depth': 3, 'max_bin': 100, 'eta': 0.37120715124988873, 'colsample_bytree': 0.8740965814602877, 'min_child_weight': 447, 'n_estimators': 570, 'subsample': 0.8699495978039323, 'reg_lambda': 0.9467441254942246, 'reg_alpha': 0.9985632742875157}.
[I 2019-09-28 05:24:54,216] Finished trial#1 resulted in value: 0.09911139908040889. Current best value is 0.09911139908040889 with parameters: {'max_depth': 3, 'max_bin': 83, 'eta': 0.3307228625386951, 'colsample_bytree': 0.7318626762226965, 'min_child_weight': 267, 'n_estimators': 889, 'subsample': 0.7888124034688451, 'reg_lambda': 0.4379399066516354, 'reg_alpha': 0.0647150119334696}.
[I 2019-09-28 05:25:36,895] Finished trial#2 resulted in value: 0.10179131470550273. Current best value is 0.09911139908040889 with parameters: {'max_depth': 3, 'max_bin': 83, 'eta': 0.3307228625386951, 'colsample_bytr

[I 2019-09-28 05:43:39,061] Finished trial#21 resulted in value: 0.08094017108454699. Current best value is 0.08094017108454699 with parameters: {'max_depth': 8, 'max_bin': 39, 'eta': 0.15672944460275628, 'colsample_bytree': 0.9527499411874936, 'min_child_weight': 282, 'n_estimators': 992, 'subsample': 0.9531093024522959, 'reg_lambda': 0.5270834076230485, 'reg_alpha': 0.14342793208805482}.
[I 2019-09-28 05:44:46,311] Finished trial#22 resulted in value: 0.08082510847263569. Current best value is 0.08082510847263569 with parameters: {'max_depth': 8, 'max_bin': 36, 'eta': 0.013842538122854492, 'colsample_bytree': 0.9148398860674646, 'min_child_weight': 262, 'n_estimators': 993, 'subsample': 0.9537037327614076, 'reg_lambda': 0.5147777429924808, 'reg_alpha': 0.14685306878712465}.
[I 2019-09-28 05:45:44,985] Finished trial#23 resulted in value: 0.08019823368266821. Current best value is 0.08019823368266821 with parameters: {'max_depth': 7, 'max_bin': 34, 'eta': 0.021340585569439248, 'colsam

[I 2019-09-28 06:02:50,736] Finished trial#42 resulted in value: 0.08070064115920417. Current best value is 0.07946323381021148 with parameters: {'max_depth': 7, 'max_bin': 46, 'eta': 0.10356436288738127, 'colsample_bytree': 0.6948730944243926, 'min_child_weight': 200, 'n_estimators': 984, 'subsample': 0.8983338102777682, 'reg_lambda': 0.571019567310019, 'reg_alpha': 0.29225575928263936}.
[I 2019-09-28 06:03:53,769] Finished trial#43 resulted in value: 0.08055531861330778. Current best value is 0.07946323381021148 with parameters: {'max_depth': 7, 'max_bin': 46, 'eta': 0.10356436288738127, 'colsample_bytree': 0.6948730944243926, 'min_child_weight': 200, 'n_estimators': 984, 'subsample': 0.8983338102777682, 'reg_lambda': 0.571019567310019, 'reg_alpha': 0.29225575928263936}.
[I 2019-09-28 06:04:53,467] Finished trial#44 resulted in value: 0.08057805238527682. Current best value is 0.07946323381021148 with parameters: {'max_depth': 7, 'max_bin': 46, 'eta': 0.10356436288738127, 'colsample_

[I 2019-09-28 06:22:49,521] Finished trial#63 resulted in value: 0.0796504295018381. Current best value is 0.07946323381021148 with parameters: {'max_depth': 7, 'max_bin': 46, 'eta': 0.10356436288738127, 'colsample_bytree': 0.6948730944243926, 'min_child_weight': 200, 'n_estimators': 984, 'subsample': 0.8983338102777682, 'reg_lambda': 0.571019567310019, 'reg_alpha': 0.29225575928263936}.
[I 2019-09-28 06:23:59,500] Finished trial#64 resulted in value: 0.08001182252482597. Current best value is 0.07946323381021148 with parameters: {'max_depth': 7, 'max_bin': 46, 'eta': 0.10356436288738127, 'colsample_bytree': 0.6948730944243926, 'min_child_weight': 200, 'n_estimators': 984, 'subsample': 0.8983338102777682, 'reg_lambda': 0.571019567310019, 'reg_alpha': 0.29225575928263936}.
[I 2019-09-28 06:25:10,750] Finished trial#65 resulted in value: 0.07985296185631918. Current best value is 0.07946323381021148 with parameters: {'max_depth': 7, 'max_bin': 46, 'eta': 0.10356436288738127, 'colsample_b

[I 2019-09-28 06:48:22,536] Finished trial#84 resulted in value: 0.08092050257484651. Current best value is 0.07865594004823462 with parameters: {'max_depth': 8, 'max_bin': 58, 'eta': 0.128184058252152, 'colsample_bytree': 0.6076130728926892, 'min_child_weight': 214, 'n_estimators': 936, 'subsample': 0.847202164170657, 'reg_lambda': 0.5346947510716556, 'reg_alpha': 0.09328364899638401}.
[I 2019-09-28 06:49:39,473] Finished trial#85 resulted in value: 0.07884763595890232. Current best value is 0.07865594004823462 with parameters: {'max_depth': 8, 'max_bin': 58, 'eta': 0.128184058252152, 'colsample_bytree': 0.6076130728926892, 'min_child_weight': 214, 'n_estimators': 936, 'subsample': 0.847202164170657, 'reg_lambda': 0.5346947510716556, 'reg_alpha': 0.09328364899638401}.
[I 2019-09-28 06:50:53,941] Finished trial#86 resulted in value: 0.07928188548976234. Current best value is 0.07865594004823462 with parameters: {'max_depth': 8, 'max_bin': 58, 'eta': 0.128184058252152, 'colsample_bytree

[I 2019-09-28 07:13:10,041] Finished trial#105 resulted in value: 0.07934185191760457. Current best value is 0.07754713470609487 with parameters: {'max_depth': 9, 'max_bin': 58, 'eta': 0.1050978537360572, 'colsample_bytree': 0.6457063337158211, 'min_child_weight': 212, 'n_estimators': 863, 'subsample': 0.8547734216821684, 'reg_lambda': 0.44605715604013324, 'reg_alpha': 0.04370520638547596}.
[I 2019-09-28 07:14:25,436] Finished trial#106 resulted in value: 0.08059958042958337. Current best value is 0.07754713470609487 with parameters: {'max_depth': 9, 'max_bin': 58, 'eta': 0.1050978537360572, 'colsample_bytree': 0.6457063337158211, 'min_child_weight': 212, 'n_estimators': 863, 'subsample': 0.8547734216821684, 'reg_lambda': 0.44605715604013324, 'reg_alpha': 0.04370520638547596}.
[I 2019-09-28 07:15:37,595] Finished trial#107 resulted in value: 0.07870306956879536. Current best value is 0.07754713470609487 with parameters: {'max_depth': 9, 'max_bin': 58, 'eta': 0.1050978537360572, 'colsam

[I 2019-09-28 07:41:21,090] Finished trial#126 resulted in value: 0.07848610876277318. Current best value is 0.07747799996717719 with parameters: {'max_depth': 9, 'max_bin': 85, 'eta': 0.061931319723249736, 'colsample_bytree': 0.6734756113357574, 'min_child_weight': 201, 'n_estimators': 876, 'subsample': 0.9034546162780236, 'reg_lambda': 0.3827759670220013, 'reg_alpha': 0.009454093649834298}.
[I 2019-09-28 07:42:50,041] Finished trial#127 resulted in value: 0.07726357574721743. Current best value is 0.07726357574721743 with parameters: {'max_depth': 9, 'max_bin': 89, 'eta': 0.061622110660508156, 'colsample_bytree': 0.6678390849666717, 'min_child_weight': 200, 'n_estimators': 879, 'subsample': 0.8982330502832879, 'reg_lambda': 0.3927315006621382, 'reg_alpha': 0.0027966233218265336}.
[I 2019-09-28 07:44:16,283] Finished trial#128 resulted in value: 0.07728442807473168. Current best value is 0.07726357574721743 with parameters: {'max_depth': 9, 'max_bin': 89, 'eta': 0.061622110660508156, 

[I 2019-09-28 08:13:06,034] Finished trial#147 resulted in value: 0.07670187090345881. Current best value is 0.07670187090345881 with parameters: {'max_depth': 9, 'max_bin': 84, 'eta': 0.043453650346884566, 'colsample_bytree': 0.6907202582733285, 'min_child_weight': 200, 'n_estimators': 934, 'subsample': 0.9284060032802744, 'reg_lambda': 0.33351731309339305, 'reg_alpha': 0.8600956402037366}.
[I 2019-09-28 08:14:39,747] Finished trial#148 resulted in value: 0.07714731487180149. Current best value is 0.07670187090345881 with parameters: {'max_depth': 9, 'max_bin': 84, 'eta': 0.043453650346884566, 'colsample_bytree': 0.6907202582733285, 'min_child_weight': 200, 'n_estimators': 934, 'subsample': 0.9284060032802744, 'reg_lambda': 0.33351731309339305, 'reg_alpha': 0.8600956402037366}.
[I 2019-09-28 08:16:12,168] Finished trial#149 resulted in value: 0.07801607671687316. Current best value is 0.07670187090345881 with parameters: {'max_depth': 9, 'max_bin': 84, 'eta': 0.043453650346884566, 'co

[I 2019-09-28 08:44:57,293] Finished trial#168 resulted in value: 0.07729783817315106. Current best value is 0.07645363883512868 with parameters: {'max_depth': 9, 'max_bin': 84, 'eta': 0.04416196906579485, 'colsample_bytree': 0.6779011782413213, 'min_child_weight': 200, 'n_estimators': 946, 'subsample': 0.9217489306438194, 'reg_lambda': 0.3338931660088901, 'reg_alpha': 0.8755106548738071}.
[I 2019-09-28 08:46:29,089] Finished trial#169 resulted in value: 0.07771462244851746. Current best value is 0.07645363883512868 with parameters: {'max_depth': 9, 'max_bin': 84, 'eta': 0.04416196906579485, 'colsample_bytree': 0.6779011782413213, 'min_child_weight': 200, 'n_estimators': 946, 'subsample': 0.9217489306438194, 'reg_lambda': 0.3338931660088901, 'reg_alpha': 0.8755106548738071}.
[I 2019-09-28 08:48:01,247] Finished trial#170 resulted in value: 0.07770531852811491. Current best value is 0.07645363883512868 with parameters: {'max_depth': 9, 'max_bin': 84, 'eta': 0.04416196906579485, 'colsamp

[I 2019-09-28 09:17:52,821] Finished trial#189 resulted in value: 0.0776221231705988. Current best value is 0.07605777598307784 with parameters: {'max_depth': 9, 'max_bin': 76, 'eta': 0.04731054942785232, 'colsample_bytree': 0.7164419586986482, 'min_child_weight': 200, 'n_estimators': 999, 'subsample': 0.9201527698999813, 'reg_lambda': 0.23470081227439588, 'reg_alpha': 0.7426408537589384}.
[I 2019-09-28 09:19:25,145] Finished trial#190 resulted in value: 0.07764865107272823. Current best value is 0.07605777598307784 with parameters: {'max_depth': 9, 'max_bin': 76, 'eta': 0.04731054942785232, 'colsample_bytree': 0.7164419586986482, 'min_child_weight': 200, 'n_estimators': 999, 'subsample': 0.9201527698999813, 'reg_lambda': 0.23470081227439588, 'reg_alpha': 0.7426408537589384}.
[I 2019-09-28 09:20:54,896] Finished trial#191 resulted in value: 0.07729846099771231. Current best value is 0.07605777598307784 with parameters: {'max_depth': 9, 'max_bin': 76, 'eta': 0.04731054942785232, 'colsam

In [36]:
import sys
sys.path.append("..") # Adds higher directory to python modules path.
from kaggleKID.preprocessing import check_dataframe
from kaggleKID.train import lgbm_process

In [39]:
lgbm_params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47
         }

In [41]:
ts_folds = TimeSeriesSplit(n_splits=5)

In [42]:
his, df_feature_importances, df_valid_pred, df_test_pred, val_metric = lgbm_process(lgbm_params, ts_folds, df_train,df_test, select_columns, is_output_feature_importance=0, verbose=1)

Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.999954	valid_1's auc: 0.909881
Early stopping, best iteration is:
[911]	training's auc: 0.999906	valid_1's auc: 0.910163
Fold 1 finished in 0:02:54.371103
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.999746	valid_1's auc: 0.927103
Early stopping, best iteration is:
[859]	training's auc: 0.999428	valid_1's auc: 0.92735
Fold 2 finished in 0:03:57.819206
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.998632	valid_1's auc: 0.92117
Early stopping, best iteration is:
[856]	training's auc: 0.997561	valid_1's auc: 0.921287
Fold 3 finished in 0:04:55.668308
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.996443	valid_1's auc: 0.936836
Early stopping, best iteration is:
[1232]	training's auc: 0.998152	valid_1's auc: 0.937163
Fold 4 finished in 0:08:24.583706
Training until validation sco